In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# === Step 1: Load Generator Model (LLM) ===
llm_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
llm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to("cuda" if torch.cuda.is_available() else "cpu")

# === Step 2: Load Retriever Model ===
retriever_model = SentenceTransformer("all-MiniLM-L6-v2")

# === Step 3: Corpus (Fake News Articles from GAN Output or Dataset) ===
news_articles = [
    "Party A is a progressive force. It stands for development, reform, and strong leadership. Their agenda emphasizes growth and justice. Many youths show increasing support. Some critics claim the policy is one-sided.",
    "The government unveiled a new economic strategy. It claims to reduce inflation and create jobs. Experts have mixed reactions. Social media spreads both facts and false claims. Citizens remain divided in opinion.",
    "Recent rally sparks controversy across the country. Slogans target emotional sentiments. Supporters see hope while others call it misleading. Analysts highlight potential misinformation. Public discourse is at an all-time high.",
    "Freedom and liberty were the core themes of the new campaign. The speech captivated a huge crowd. Opposition parties criticized it for lacking substance. Debates surged online. Authenticity of claims remains under scrutiny.",
    "Critics say the new reforms are biased. Youth-targeted ads are misleading. Many posts spread half-truths. Government denies allegations. Media houses cover both sides intensely."
]

# === Step 4: Create FAISS Index ===
news_embeddings = retriever_model.encode(news_articles, convert_to_tensor=False)
embedding_dim = news_embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(news_embeddings))

# === Step 5: RAG Chatbot Function ===
def rag_chatbot(user_query, top_k=2):
    query_embedding = retriever_model.encode([user_query])[0]
    _, indices = index.search(np.array([query_embedding]), top_k)
    retrieved_docs = [news_articles[i] for i in indices[0]]

    context = " ".join(retrieved_docs)
    prompt = f"Context: {context}\n\nQuestion: {user_query}\nAnswer:"
    inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(llm_model.device)
    
    with torch.no_grad():
        outputs = llm_model.generate(**inputs, max_new_tokens=150)
    
    answer = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# === Chat Loop ===
if __name__ == "__main__":
    print("🤖 RAG Fake News Chatbot Ready! Ask about any news topic.")
    while True:
        user_input = input("\n🧑 You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Goodbye!")
            break
        response = rag_chatbot(user_input)
        print(f"📰 Chatbot: {response}")


🤖 RAG Fake News Chatbot Ready! Ask about any news topic.
